### 데이터 수집
    * 스크래이핑을 통한 수집
        - 뉴스
        - SNS
        - OTT
        - 댓글
    * json api를 활용
        - 핀테크 금융
        - 프롭테크 부동산
        - 마케팅
        - SCM
        - CRM
    * 빅데이터 생성
    * DB 활용
    * 시각화
    * 사무 자동화
        - 오피스
        - RPA
    * 사회소통능력 
        - 유튜브
        - SNS
        - 홈페이지 관리
### 개별 데이터 분석
    * 정형 데이터 분석: 개별분석
        - 모멘텀 분석
        - 기초 분석
    * 비정형 데이터 분석: 개별분석
        - Count Vectorize
        - TF-IDF
        - Word2Vec:cbow
        - LDA

### 데이터 융합
    * 정형데이터: 주가 데이터
    * 비정형 데이터: 뉴스 데이터
    * 데이터 융합
        - 데이터베이스 융합: DB/SQL join
        - Pandas 활용: Merge 명령어 사용
### 고급 데이터 처리 기법
*  해당분야 용어 기법
* 

In [1]:
### 기초데이터: DB join 방식 활용
### count 벡터화: Pandas 활용
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3

In [38]:
dbname='./navernews.db'
def getDB(sql='select nidx,ndate from nnews where nday is NULL'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #sql='select * from stockp where skey>20200101 order by skey asc'
        qry=cur.execute(sql)
        desc=[row[0] for row in qry.description]
        data=qry.fetchall()
    return((desc,data))

In [39]:
def dbNday(ndays):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #sql='select * from stockp where skey>20200101 order by skey asc'
        sql='update  nnews set nday="'+ndays[1]+'" where nidx = '+str(ndays[0])
        qry=cur.execute(sql)
    return(qry)

In [40]:
def updateNdays():
    res=getDB()
    ndays=[]
    for r in res[1]:
        ndays.append((r[0],r[1][:10]))
    for n in ndays:
        print(dbNday(n))

In [50]:
updateNdays()

In [57]:
sql= "select sday,sopen,shigh,slow,sclose,svol,count(nidx) nd,group_concat(ntitle) gtitle from stockp a left outer join nnews b on a.sday=b.nday group by sday"
mdata=getDB(sql)

In [59]:
df=pd.DataFrame(data=mdata[1],columns=mdata[0])
df.tail()

,sday,sopen,shigh,slow,sclose,svol,nd,gtitle
885,2021-03-30,276500,279500,276000,279000,41692,0,None
886,2021-03-31,279000,281500,276500,281000,58150,0,None
887,2021-04-01,282500,283000,277500,279000,72605,0,None
888,2021-04-02,278000,278500,272000,274000,119686,0,None
889,2021-04-05,275000,277500,274000,276000,17034,0,None


In [61]:
nlpdf=df[['sday','gtitle']].copy()
nlpdf.head()

,sday,gtitle
0,2017-08-17,None
1,2017-08-18,None
2,2017-08-21,None
3,2017-08-22,None
4,2017-08-23,None


In [62]:
from konlpy.tag import Kkma
kkma=Kkma()

def dictsort(r):
    sr=sorted(r.items(),key=(lambda x:x[1]),reverse=True)
    return r

def getPOS(rres):
    p=kkma.pos(rres)
    dictpos={}
    words=[]
    #표제어 추출
    check=['NNG','NNP','NNB','NNM','VV','VA','VXV','VXA','VCP','VCN','SF','EFN','EFQ']
    for  w,pos in p:
        if(pos in check):
            words.append(w)
            if (w in dictpos.keys()):
                dictpos[w]+=1
            else:
                dictpos[w]=1
    return (dictpos,words)  

In [64]:
gtitles=list(nlpdf['gtitle'])
gtitles

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [65]:
corpus=gtitles
# 카운트 기반 TDM
from sklearn.feature_extraction.text import CountVectorizer
vect=CountVectorizer()
tdm=vect.fit_transform(corpus).toarray()
tdm.shape


AttributeError: 'NoneType' object has no attribute 'lower'

In [63]:
getp=getPOS(docs[0][0])
sen=' '.join(getp[1])
sen

NameError: name 'docs' is not defined